Spousteni pyspark

`pyspark --num-executors 2 --executor-memory 1500M --packages com.databricks:spark-csv_2.10:1.5.0 --master yarn`

In [ ]:
# uzitecne importy

from pyspark.sql.types import *
from pyspark.sql import functions as F
import re
from itertools import cycle

1 Načtěte soubor se stopwords.

In [ ]:
stopw = sc.textFile("/user/pascepet/stopwords.txt").collect()
stopw = set(stopw)

2 Načtěte písňové texty z CSV přímo do DataFrame s definicí schématu, nakešujte:

cesta `/user/pascepet/lyrics_data/`

oddelovac `','`

schema

```
id – long
album – string
rok – integer
interpret – string
zanr – string
text – string
```

In [ ]:
schema_lyrics = StructType([
    StructField("id", LongType(), True),
    StructField("album", StringType(), True),
    StructField("rok", IntegerType(), True),
    StructField("interpret", StringType(), True),
    StructField("zanr", StringType(), True),
    StructField("text", StringType(), True)])

# a nyni nacteni souboru
songsDF = sqlContext.read \
    .format("com.databricks.spark.csv") \
    .option("header", "false") \
    .option("delimiter", ",") \
    .schema(schema_lyrics) \
    .load("/user/pascepet/lyrics_data/") \
    .cache()

3 Přidejte pole `poc_slov`, které bude obsahovat počet slov v textu příslušné písně (slovo je část řetězce mezi dvěma mezerami). 
Obtížnější varianta: přidejte pole `pocet_slov_unik`, které bude obsahovat počet unikátních slov po převedení na malá písmena a vyhození znaků `,.:;!?()[]`


In [ ]:
songsDF = songsDF.withColumn('poc_slov', F.size(F.split(songsDF.text, ' ')))

songsDF = songsDF.withColumn('text_upr', F.lower(F.regexp_replace(songsDF.text, '[,.;:?!()\[\]]', '')))  # na mala pismena, procistit
songsDF = songsDF.withColumn('text_upr2', F.split(songsDF.text_upr, ' '))  # rozdelit na slova => v poli je array jednotlivych slov
countUniq = F.udf(lambda a: len(set(a))) # pocet unikatnich - abychom mohli pouzit funkce Pythonu, musime definovat uzivatelskou funkci
songsDF = songsDF.withColumn('poc_slov_unik', countUniq(songsDF.text_upr2))  # ... a tu aplikovat na array ve sloupci
songsDF = songsDF.drop('text_upr').drop('text_upr2')  # vyhodime pracovni sloupce

songsDF.cache() # nove nakesovani - nepovinne, ale uzitecne

4 Spočtěte za každého interpreta průměrný počet slov v textu a vypište 20 interpretů s nejvyšším průměrem, ale jen takových, kteří mají v datech aspoň 10 písní.


In [ ]:
songsDF.registerTempTable("songs")
sqlContext.sql("""select interpret, avg(poc_slov) as prum_slov, count(*) as poc_pisni from songs
    group by interpret
    order by prum_slov desc
    """).filter("poc_pisni>=10").show(20)

5 Zjistěte, kteří interpreti mají v datech aspoň 500 písní.

In [ ]:
interprets = songsDF.groupBy('interpret').count() \
    .toDF('interpret', 'pocet').filter("pocet >= 500")

interprets.show()

6 Najděte 20 nejčastěji se vyskytujících slov, která nejsou v stopwords. Každé slovo se počítá tolikrát, kolikrát v textu je. (Obtížnější varianta: každé slovo se počítá nejvýše jednou za každou píseň.)

In [ ]:
words_top = songsDF.flatMap(lambda r: strClean(r[5]).split(" ")) \
    .filter(lambda r: r not in stopw) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda r: r[1], False)

words_top.take(20)

7 Vybrat z nejčastěji se vyskytujících slov pět charakteristických a k DataFrame s písněmi přidejte pro každé vybrané slovo příznak, zda se dané slovo v dané písni vyskytuje (0/1 nebo False/True).

In [ ]:
words_top_dist = songsDF.flatMap(lambda r: list(set(strClean(r[5]).split(" ")))) \
    .filter(lambda r: r not in stopw) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda r: r[1], False)

words_top_dist.take(20)


songsDF = songsDF.withColumn('text_lower',
                             F.lower('text'))  # pripoji se sloupec s textem transformovanym na mala pismena
songsDF = songsDF.withColumn('is_love', F.when(F.regexp_extract('text', '\W(love)\W', 1) == 'love', 1).otherwise(0))
songsDF = songsDF.withColumn('is_never', F.when(F.regexp_extract('text', '\W(never)\W', 1) == 'never', 1).otherwise(0))
songsDF = songsDF.withColumn('is_back', F.when(F.regexp_extract('text', '\W(back)\W', 1) == 'back', 1).otherwise(0))
songsDF = songsDF.withColumn('is_say', F.when(F.regexp_extract('text', '\W(say)\W', 1) == 'say', 1).otherwise(0))
songsDF = songsDF.withColumn('is_now', F.when(F.regexp_extract('text', '\W(now)\W', 1) == 'now', 1).otherwise(0))
songsDF = songsDF.drop('text_lower')  # pomocny sloupec se zase vyhodi

8 Vyberte z interpretů s nejvyššími zastoupením pět a pro každého spočtěte podíl výskytu každého z pěti slov vybraných výše (podíl = počet písní s výskytem slova / počet všech písní interpreta). Najděte dvojice interpretů s nejpodobnějšími a nejméně podobnými podíly.

In [ ]:
songsDF = songsDF.withColumn('dummy1', F.lit(1))  # vsude se prida jednicka - jejich souctem vznikne pocet
interprets_words = songsDF.select('interpret', 'dummy1', 'is_love', 'is_never', 'is_back', 'is_say', 'is_now') \
    .filter("interpret in ('bob-dylan', 'eminem', 'elton-john', 'frank-zappa', 'celine-dion')") \
    .groupBy('interpret') \
    .sum() \
    .toDF('interpret', 'pocet', 'pocet_love', 'pocet_never', 'pocet_back', 'pocet_say', 'pocet_now')

interprets_words = interprets_words.withColumn('podil_love',
                                               F.round(interprets_words.pocet_love / interprets_words.pocet, 3)) \
    .withColumn('podil_never', F.round(interprets_words.pocet_never / interprets_words.pocet, 3)) \
    .withColumn('podil_back', F.round(interprets_words.pocet_back / interprets_words.pocet, 3)) \
    .withColumn('podil_say', F.round(interprets_words.pocet_say / interprets_words.pocet, 3)) \
    .withColumn('podil_now', F.round(interprets_words.pocet_now / interprets_words.pocet, 3)) \
    .select('interpret', 'podil_love', 'podil_never', 'podil_back', 'podil_say', 'podil_now')

interprets_words.show()